In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import pickle
import string
import re
import string
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, auc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import IterativeImputer
import pickle

In [2]:
data_pre = pd.read_pickle('Cyclical_encoded_data.pkl')

In [3]:
def set_min_value(df):
    values = []
    for i in range(len(df.columns) - 2):
        values.append(-np.inf)
    values.insert(40,1) 
    values.pop()
    return values

def set_max_value(df):
    values = []
    for i in range(len(df.columns) - 2):
        values.append(np.inf)
    values.insert(40,4) 
    values.pop()
    return values 

In [ ]:
print(data_pre.columns)
print(len(data_pre.columns))

In [ ]:
min_values = set_min_value(data_pre)
max_values = set_max_value(data_pre)

In [ ]:
imp_forest = IterativeImputer(ExtraTreesRegressor(n_estimators=10, random_state=0)
, max_iter=100, random_state=0, min_value=min_values, max_value=max_values)
data = data_pre.drop(['RHoMIS_ID','ID_HH'],axis=1)
# imp.fit(data)
# data = imp.transform(data)
data_imputed_forest = pd.DataFrame(imp_forest.fit_transform(data),columns = data.columns)

In [ ]:
imp_bayes = IterativeImputer(max_iter=100, random_state=0, min_value=min_values, max_value=max_values)
# data = data_pre.drop(['RHoMIS_ID','ID_HH'],axis=1)
# imp.fit(data)
# data = imp.transform(data)
data_imputed_bayes = pd.DataFrame(imp_bayes.fit_transform(data),columns = data.columns)

In [ ]:
imp_decision_tree = IterativeImputer(DecisionTreeRegressor(max_features='sqrt', random_state=0), max_iter=100, random_state=0, min_value=min_values, max_value=max_values)
# data = data_pre.drop(['RHoMIS_ID','ID_HH'],axis=1)
# imp.fit(data)
# data = imp.transform(data)
data_imputed_decision_tree = pd.DataFrame(imp_decision_tree.fit_transform(data),columns = data.columns)

In [ ]:
imp_knn = IterativeImputer(KNeighborsRegressor(n_neighbors=15)
, max_iter=100, random_state=0, min_value=min_values, max_value=max_values)
# data = data_pre.drop(['RHoMIS_ID','ID_HH'],axis=1)
# imp.fit(data)
# data = imp.transform(data)
data_imputed_knn = pd.DataFrame(imp_knn.fit_transform(data),columns = data.columns)

In [4]:
data_imputed_bayes['Food_InsecurityLevel'] = data_imputed_bayes['Food_InsecurityLevel'].astype('int32')
data_imputed_knn['Food_InsecurityLevel'] = data_imputed_knn['Food_InsecurityLevel'].astype('int32')
data_imputed_decision_tree['Food_InsecurityLevel'] = data_imputed_decision_tree['Food_InsecurityLevel'].astype('int32')
data_imputed_forest['Food_InsecurityLevel'] = data_imputed_forest['Food_InsecurityLevel'].astype('int32')

NameError: name 'data_imputed_bayes' is not defined

In [ ]:
data_imputed_bayes.to_pickle('imputed_data_bayes.pkl')

In [ ]:
data_imputed_knn.to_pickle('imputed_data_knn.pkl')

In [ ]:
data_imputed_decision_tree.to_pickle('imputed_data_decision_tree.pkl')

In [ ]:
data_imputed_forest.to_pickle('imputed_data_forest.pkl')

In [ ]:
print(data_imputed['Food_InsecurityLevel'].unique())

In [ ]:
data_pre.describe()

In [19]:
def multiple_imputation(dfs):
    final_df = pd.DataFrame(dfs[0])
    for i in range(len(dfs) - 1):
        final_df = final_df + dfs[i] 
    final_df = final_df/len(dfs)    
    return final_df        

            
# def average_columns(dfs):
#     df = pd.DataFrame(dfs[0][column])
#     for i in range (len(dfs) - 1):
#         df = df + dfs[i + 1][column]
#     df = df / len(dfs) 
#     return df

In [20]:
data_imputed_forest = pd.read_pickle('imputed_data_forest.pkl')
data_imputed_bayes = pd.read_pickle('imputed_data_bayes.pkl')
data_imputed_decision_tree = pd.read_pickle('imputed_data_decision_tree.pkl')
data_imputed_knn = pd.read_pickle('imputed_data_knn.pkl')

dfs = []
dfs.append(data_imputed_bayes)
dfs.append(data_imputed_forest)
dfs.append(data_imputed_decision_tree)
dfs.append(data_imputed_knn)

In [51]:
multiple_imputed_data = multiple_imputation(dfs)

multiple_imputed_data.describe()

In [52]:
multiple_imputed_data.describe()

,YEAR,ITERATION,GPS_LAT,GPS_LON,GPS_ALT,HHsizemembers,HHsizeMAE,LandOwned,LandCultivated,LivestockHoldings,...,Country_<Palestine>,Country_<Peru>,Country_<Rwanda>,Country_<Senegal>,Country_<Sierra Leone>,Country_<South Africa>,Country_<Tanzania>,Country_<Uganda>,Country_<Vietnam>,Country_<Zambia>
count,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,...,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000
mean,2018.181576,1.028859,11.728378,25.881654,1060.663581,10.468091,8.709608,11.985390,7.462824,14.821882,...,0.006942,0.004786,0.075939,0.004618,0.013156,0.003695,0.080614,0.033281,0.043470,0.033393
std,1.277014,0.167411,26.515093,37.772153,3914.830892,635.207291,613.475805,361.579639,214.433713,2019.286406,...,0.083028,0.069019,0.264904,0.067803,0.113943,0.060673,0.272245,0.179372,0.203915,0.179663
min,2015.000000,1.000000,-78.745000,-446.327871,-164500.705935,1.000000,0.500000,-34197.893571,-5.462145,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.000000,1.000000,-2.210000,-0.080000,318.672030,4.000000,2.930000,0.937500,0.750000,0.120000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2018.000000,1.000000,7.860000,29.195000,946.974689,6.000000,4.470000,2.500000,1.600000,1.215000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2019.000000,1.000000,13.450000,36.760000,1650.000000,8.000000,6.500085,6.000000,3.500000,3.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2020.000000,2.000000,108.243000,179.411759,366653.752325,119997.000000,115897.410000,22000.000000,30000.000000,381663.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [53]:
multiple_imputed_data['HHsizemembers'] = multiple_imputed_data['HHsizemembers'].astype('int32')
multiple_imputed_data['HHsizemembers'].unique()

array([     7,     10,      6,      8,     11,      5,      9,     12,
            4,     13,      2,      1,     20,     16,     32,     24,
           28,     15,     30,     23,     19,     17,      3,     21,
           27,     14,     31,     18,     25,     29,     22,     44,
           26,     34,     86,     52,     62,     96,     68,     40,
           53,     80,     70,     51,     48,     33,     60,     67,
           37,     54,     71,     41,     42,     38,     55,     91,
           35,     45,     39,     61,     36,     43,     50,     64,
          112,     99, 119997,     84,     56,     46,     73,     72,
           65,     94,     85,     47,    119,    116,    117,     69,
           75,    417,     66,     90,     79,     92,     59,     49,
          369,     88,     77,     81,     76,    115,     58,     57,
          111,    109,     63,    148,    133,     74,     78,   3984],
      dtype=int32)

In [54]:
def replace_categoricals(data_p, data_i):
    for column in data_i.columns:
        if '<' in column:
            data_i[column] = data_p[column]
    return data_i    
        
    

In [59]:
data_encoded = pd.read_pickle("Cyclical_encoded_data.pkl")
data_encoded.describe()

,YEAR,ITERATION,GPS_LAT,GPS_LON,GPS_ALT,HHsizemembers,HHsizeMAE,LandOwned,LandCultivated,LivestockHoldings,...,Country_<Palestine>,Country_<Peru>,Country_<Rwanda>,Country_<Senegal>,Country_<Sierra Leone>,Country_<South Africa>,Country_<Tanzania>,Country_<Uganda>,Country_<Vietnam>,Country_<Zambia>
count,35726.000000,35726.000000,34688.000000,34688.000000,27761.000000,34763.000000,34763.000000,26235.000000,33862.000000,35726.000000,...,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000
mean,2018.181576,1.028859,11.723959,23.046392,940.749333,10.462388,8.734048,13.159772,7.383673,14.821882,...,0.006942,0.004786,0.075939,0.004618,0.013156,0.003695,0.080614,0.033281,0.043470,0.033393
std,1.277014,0.167411,26.864828,34.239292,786.583853,643.944947,621.915028,295.799118,218.623158,2019.286406,...,0.083028,0.069019,0.264904,0.067803,0.113943,0.060673,0.272245,0.179372,0.203915,0.179663
min,2015.000000,1.000000,-78.745000,-89.690000,-1932.000000,1.000000,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.000000,1.000000,-2.260000,-0.287500,306.600000,4.000000,2.860000,0.750000,0.800000,0.120000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2018.000000,1.000000,7.830000,29.020000,755.500000,6.000000,4.390000,1.875000,1.600000,1.215000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2019.000000,1.000000,13.370000,35.770000,1473.250000,8.000000,6.310000,4.000000,3.200000,3.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2020.000000,2.000000,108.243000,106.040000,8234.210000,119997.000000,115897.410000,22000.000000,30000.000000,381663.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [60]:
multiple_imputed_data.describe()

,YEAR,ITERATION,GPS_LAT,GPS_LON,GPS_ALT,HHsizemembers,HHsizeMAE,LandOwned,LandCultivated,LivestockHoldings,...,Country_<Palestine>,Country_<Peru>,Country_<Rwanda>,Country_<Senegal>,Country_<Sierra Leone>,Country_<South Africa>,Country_<Tanzania>,Country_<Uganda>,Country_<Vietnam>,Country_<Zambia>
count,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,...,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000,35726.000000
mean,2018.181576,1.028859,11.728378,25.881654,1060.663581,10.454935,8.709608,11.985390,7.462824,14.821882,...,0.006942,0.004786,0.075939,0.004618,0.013156,0.003695,0.080614,0.033281,0.043470,0.033393
std,1.277014,0.167411,26.515093,37.772153,3914.830892,635.207291,613.475805,361.579639,214.433713,2019.286406,...,0.083028,0.069019,0.264904,0.067803,0.113943,0.060673,0.272245,0.179372,0.203915,0.179663
min,2015.000000,1.000000,-78.745000,-446.327871,-164500.705935,1.000000,0.500000,-34197.893571,-5.462145,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.000000,1.000000,-2.210000,-0.080000,318.672030,4.000000,2.930000,0.937500,0.750000,0.120000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2018.000000,1.000000,7.860000,29.195000,946.974689,6.000000,4.470000,2.500000,1.600000,1.215000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2019.000000,1.000000,13.450000,36.760000,1650.000000,8.000000,6.500085,6.000000,3.500000,3.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2020.000000,2.000000,108.243000,179.411759,366653.752325,119997.000000,115897.410000,22000.000000,30000.000000,381663.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [61]:
multiple_imputed_data.to_pickle("multiple_imputed_data.pkl")